- Genie Top200 Chart

In [1]:
import requests
import pandas as pd

### 1. 인터넷에서 데이터 가져오기

In [2]:
# Genie top200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)

In [3]:
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [4]:
# Chrome User-Agent
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}
req = requests.get(url, headers=header)
html = req.text
type(html)

str

In [5]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

### 찾으려고 하는 데이터의 노드 찾기 

In [6]:
trs = soup.find_all('tr')[1:]
len(trs)

50

In [7]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [8]:
# 개발자 도구에서 찾아주는 Selector
tbody = soup.select_one('#body-content > div.newest-list > div > table > tbody')
trs = tbody.select('tr')
len(trs)

50

In [9]:
#   셀레니움 사용해서 이용가능 , XPath 값은 아래와 같음 
# //*[@id="body-content"]/div[6]/div/table/tbody

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출
- rank, title , artist, album

In [10]:
tr = trs[0]

In [11]:
import string
# rank
tr.select_one('.number').get_text().split('\n')[0]

'1'

In [12]:
rank = tr.select_one('.number').get_text().split('\n')[0]

In [13]:
# title 클래스니까 점 찍음 , 띄어쓰기는 또다른 클래스값을 말함 
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'Counting Stars (Feat. Beenzino)'

In [14]:
# artist
artis = tr.select_one('.artist.ellipsis').get_text().strip()
artis

"BE'O (비오)"

In [15]:
# album
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'Counting Stars'

In [16]:
album,artis,title,rank

('Counting Stars', "BE'O (비오)", 'Counting Stars (Feat. Beenzino)', '1')

### 4. 한페이지에 있는 모든 데이터를 반복문으로 가져와 데이터프레임 만들기

In [17]:
#1) 개별리스트 만들기
#2) 딕셔너리의 리스트 만들기

In [18]:
rank_list, title_list, artis_list, album_lsit = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artis = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    rank_list.append(rank)
    title_list.append(title)
    artis_list.append(artis) 
    album_lsit.append(album)

In [19]:
df = pd.DataFrame(
    {
    '제목' : title_list,
    '가수' : artis_list,
    '앨범' : album_lsit
    },
    index=rank_list
)
df

,제목,가수,앨범
1,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars
2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
3,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
4,매일 크리스마스 (Everyday Christmas),다비치,매일 크리스마스 (Everyday Christmas)
5,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
6,ELEVEN,IVE (아이브),ELEVEN
7,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
8,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
9,All I Want for Christmas Is You,Mariah Carey,Merry Christmas (Deluxe Anniversary Edition)
10,strawberry moon,아이유 (IU),strawberry moon


In [20]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artis = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank, title,artis,album])
df = pd.DataFrame(lines, columns=['순위','제목','가수','앨범'])
df.head()

,순위,제목,가수,앨범
0,1,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
3,4,매일 크리스마스 (Everyday Christmas),다비치,매일 크리스마스 (Everyday Christmas)
4,5,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3


In [21]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artis = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    d = {'순위':rank,'제목':title,'가수':artis,'앨범':album}
    lines.append(d)

df = pd.DataFrame(lines, columns=['순위','제목','가수','앨범'])
df.head()

,순위,제목,가수,앨범
0,1,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
3,4,매일 크리스마스 (Everyday Christmas),다비치,매일 크리스마스 (Everyday Christmas)
4,5,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3


### 5.모든 페이지의 데이터 가져오기 

In [22]:
lines = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')

    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artis = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        d = {'순위':rank,'제목':title,'가수':artis,'앨범':album}
        lines.append(d)
df = pd.DataFrame(lines)
df

,순위,제목,가수,앨범
0,1,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
3,4,매일 크리스마스 (Everyday Christmas),다비치,매일 크리스마스 (Everyday Christmas)
4,5,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
...,...,...,...,...
195,196,내일이 오면 (Feat. 기리보이 & BIG Naughty (서동현)),릴보이 (lIlBOI),쇼미더머니 9 Episode 3
196,197,이젠 잊기로 해요 (Drama Ver.),미도와 파라솔,슬기로운 의사생활 시즌2 OST Special (tvN 목요드라마)
197,198,추적이는 여름 비가 되어,장범준,장범준 싱글
198,199,시절인연 (時節因緣),이찬원,꼰대인턴 OST Part 2 (MBC 수목드라마)
